In [ ]:
import vantage6.client as v6client
from time import sleep
import os

In [ ]:
v6_host = 'https://' + os.environ['V6_HOST']
v6_user = os.environ['V6_USER']
v6_password = os.environ['V6_PASSWORD']
v6_key = os.environ['V6_KEY']

In [ ]:
client = v6client.Client(v6_host, None)

client.authenticate(v6_user, v6_password)
client.setup_encryption(v6_key)

In [ ]:
nodes = client.node.list(is_online=True)
nodes

In [ ]:
orgs = [n['id'] for n in nodes['data']]
orgs

In [ ]:
verticox_image = 'harbor.carrier-mu.src.surf-hosted.nl/carrier/verticox'



input_params = {'method': 'verticox', 'master': True, 'kwargs':
                {
                    'feature_columns': ['afb', 'age',
#                                         'av3', 'bmi', 'chf', 'cvd', 
                    #                     'diasbp', 'gender', 'hr','los', 'miord',
                    #                     'mitype', 'sho', 
                                        'sysbp'],
                    'event_times_column': 'event_time',
                    'event_happened_column': 'event_happened',
                     'datanode_ids': orgs[1:],
                    'precision':0.01
                }
               }


task = client.task.create(collaboration=1, organizations=[orgs[0]], name='verticox', 
                          image=verticox_image, description='verticox test',
                   input=input_params)
                   

In [ ]:
result_id = task['results'][0]['id']

r = client.result.get(result_id)

while not r['finished_at']:
    sleep(3)
    r = client.result.get(result_id)
    
print(f'Result:\n{r["result"]}')
print(f'Log:\n{r["log"]}')
      

In [ ]:
r = client.result.get(result_id)
r['finished_at']

In [ ]:
client.task.list()['data'][0]

In [ ]:
for t in client.task.list()['data']:
    id = t['id']
    client.task.delete(id)